# Main Functions
Based on https://arxiv.org/pdf/1806.07862


In [11]:
import numpy as np

# Define physical constants
k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
h = 4.135667696e-15      # Planck constant in eV*s

def photon_occupation_number(energy, temperature):
  """
  Calculates the photon occupation number using the Bose-Einstein distribution.

  This function determines the average number of photons occupying a quantum
  state with a specific energy at a given temperature. The calculation is
  based on the principles of Bose-Einstein statistics, which apply to bosons
  like photons.

  Args:
    energy (float): The energy of the quantum state in electron volts (eV).
    temperature (float): The temperature of the system in Kelvin (K).

  Returns:
    float: The average occupation number of the state. Returns 0 if the
           temperature is non-positive. Returns np.inf if energy is 0.
  """
  if temperature <= 0:
    return 0.0
  if energy == 0 and temperature > 0:
      # For zero energy, the occupation number diverges.
      return np.inf

  # The argument for the exponential function
  exponent = energy / (k_B * temperature)

  # The Bose-Einstein distribution for photons (or other bosons)
  try:
    # np.exp can overflow for large exponents, in which case the
    # occupation number is effectively zero.
    if exponent > 700: # np.exp(709.7) is approx the max float64
        return 0.0
    occupation = 1 / (np.exp(exponent) - 1)
  except (OverflowError, ZeroDivisionError):
    # Fallback for any other math errors
    occupation = 0.0

  return occupation

def calculate_photons_after_stage(n_incident, A_dB, T_stage, photon_frequency):
  """
  Calculates the total number of photons at the output of a single stage.

  This function models an attenuator or component in a signal chain. It takes
  the incident photon number (n_{i-1}), attenuates it, and adds the thermal
  noise photons generated by the stage itself to calculate the output photon
  number (n_i).

  The formula used is: n_i = (n_{i-1} / L) + (1 - 1/L) * n_thermal

  Args:
    n_incident (float): The mean number of incident photons from the previous stage.
    A_dB (float): The attenuation of the stage in decibels (dB). Must be non-negative.
    T_stage (float): The physical temperature of the stage in Kelvin (K).
    photon_frequency (float): The frequency of the photons in Hertz (Hz).

  Returns:
    float: The mean number of photons at the output of the stage.
    tuple: A tuple containing:
      - n_incident_contribution (float): Contribution from the incident photons.
      - n_thermal_contribution (float): Contribution from the thermal noise photons.
      
    
  """
  if A_dB < 0:
      # Attenuation cannot be negative (which would imply gain)
      return n_incident

  # Convert attenuation from dB to a linear factor L, where L = P_in / P_out
  L = 10**(A_dB / 10.0)

  # Calculate the energy of the photon from its frequency
  energy = h * photon_frequency # Energy in eV

  # Calculate the mean photon number of the stage's thermal bath
  n_thermal = photon_occupation_number(energy, T_stage)

  # Attenuate the incident photons and add the noise from the current stage
  n_output = (n_incident / L) + (1 - 1/L) * n_thermal

  # incident contribution 
  n_incident_contribution = n_incident / L
  # thermal contribution
  n_thermal_contribution = (1 - 1/L) * n_thermal

  return n_output, n_incident_contribution, n_thermal_contribution



# BF5 Pippin Chain


In [30]:
# --- New Example: Modeling a full attenuation chain for a qubit ---
print("--- Attenuation Chain Calculation ---")

# Define the signal frequency (e.g., for a superconducting qubit)
qubit_freq = 5e9 # 5 GHz
photon_energy_qubit = h * qubit_freq
print(f"Signal Frequency: {qubit_freq / 1e9} GHz (Energy: {photon_energy_qubit:.3e} eV)\n")

# Initial condition: The signal starts at room temperature, so the initial
# noise is the thermal occupation at 300K. We assume no signal is present yet.
# This represents the noise from the source at the start of the chain.
n_0 = photon_occupation_number(photon_energy_qubit, 300.0)
print(f"Initial noise photons from 300K source: {n_0:.4f}")
print("-" * 40)

# Define the attenuation chain as a list of tuples: (Attenuation_dB, Temperature_K)
attenuation_chain = [
    # (20.0, 300.0), # Stage 1: Room Temperature
    (20.0, 4.0),   # Stage 2: 4K (Still-plate) stage
    (20.0, 0.1),   # Stage 3: 100mK (Cold-plate) stage
    (10.0, 0.01)   # Stage 4: 10mK (Mixing-chamber) stage, connected to the qubit
]

# Propagate the noise through the chain
n_photons = n_0
for i, (A_dB, T_stage) in enumerate(attenuation_chain):
    n_photons, n_signal_contribution, n_thermal_contribution = calculate_photons_after_stage(n_photons, A_dB, T_stage, qubit_freq)
    print(f"After Stage {i+1} ({T_stage} K, {A_dB} dB):")
    print(f"  - Output photon number = {n_photons:.6f}")
    print(f"  - Signal contribution = {n_signal_contribution:.6f}")
    print(f"  - Thermal contribution = {n_thermal_contribution:.6f}")

print("-" * 40)
print(f"Final mean photon number at the qubit: {n_photons:.6f}")
print("This is the effective thermal noise the qubit experiences from the input line.")

--- Attenuation Chain Calculation ---
Signal Frequency: 5.0 GHz (Energy: 2.068e-05 eV)

Initial noise photons from 300K source: 1249.6972
----------------------------------------
After Stage 1 (4.0 K, 20.0 dB):
  - Output photon number = 28.509523
  - Signal contribution = 12.496972
  - Thermal contribution = 16.012551
After Stage 2 (0.1 K, 20.0 dB):
  - Output photon number = 0.383907
  - Signal contribution = 0.285095
  - Thermal contribution = 0.098812
After Stage 3 (0.01 K, 10.0 dB):
  - Output photon number = 0.038391
  - Signal contribution = 0.038391
  - Thermal contribution = 0.000000
----------------------------------------
Final mean photon number at the qubit: 0.038391
This is the effective thermal noise the qubit experiences from the input line.


In [ ]:
#

# Slow vs Fast Frequency drifts 

def slow()

In [32]:
def slow(Kerr, n_th, kappa):
    # Calculate the slow noise rate
    n_slow = Kerr * np.sqrt(n_th) / kappa
    T_phi = 1/ n_slow
    return T_phi

def fast(Kerr, n_th, kappa):
    # Calculate the fast noise rate
    n_fast = Kerr**2 * n_th / kappa
    T_phi = 1/ n_fast
    return T_phi

Kerr = 6 # khz 
n_ths = [0.01, 0.001]
kappa = 1 # KHZ 
for n_th in n_ths:
    T_phi_slow = slow(Kerr, n_th, kappa)
    T_phi_fast = fast(Kerr, n_th, kappa)
    print(f"n_th: {n_th}, T_phi_slow: {T_phi_slow:.2f} ms, T_phi_fast: {T_phi_fast:.2f} ms")

n_th: 0.01, T_phi_slow: 1.67 ms, T_phi_fast: 2.78 ms
n_th: 0.001, T_phi_slow: 5.27 ms, T_phi_fast: 27.78 ms


In [ ]:
Kerr = 150000 # khz 
n_ths = [0.01, 0.001]
kappa = 0.5 # KHZ 
for n_th in n_ths:
    T_phi_slow = slow(Kerr, n_th, kappa)
    T_phi_fast = fast(Kerr, n_th, kappa)
    print(f"n_th: {n_th}, T_phi_slow: {T_phi_slow * 1e3:.4f} us, T_phi_fast: {T_phi_fast * 1e3:.4f} us")

n_th: 0.01, T_phi_slow: 0.0833 us, T_phi_fast: 0.0000 us
n_th: 0.001, T_phi_slow: 0.2635 us, T_phi_fast: 0.0001 us
